In [28]:
import pandas as pd
import numpy as np
import os
import pydicom
import matplotlib.pyplot as plt

from scipy.misc import imsave
from imageio import imwrite, imread
from tqdm import tqdm
from skimage import *
import itertools


%matplotlib inline


import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam,Adagrad
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from math import sqrt
from keras.callbacks import History 
from keras.optimizers import Adam, SGD
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,Input
from keras.layers import BatchNormalization
from keras.models import Model
from keras.activations import relu
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
from math import sqrt
from keras import backend as K
from keras.callbacks import History 
import gc

In [2]:
%run '/gpfs/cbica/home/santhosr/Codes/utils.py'

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [34]:
batchSize = 25
imageSize = (512,512)

In [35]:
train_flow = train_datagen.flow_from_directory('/gpfs/cbica/home/santhosr/Datasets/jpeg',batch_size=batchSize,target_size=imageSize)

Found 1142 images belonging to 2 classes.


In [8]:
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides)(zp)
    bn = BatchNormalization(axis=3)(conv)
    act = LeakyReLU(1/10)(bn)
    return act

In [42]:
inp = Input(shape=(imageSize[0], imageSize[1],3))

conv1 = conv_layer(inp, 64, zp_flag=False)
conv2 = conv_layer(conv1, 64, zp_flag=False)
mp1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv2)
# 23
conv3 = conv_layer(mp1, 128, zp_flag=False)
conv4 = conv_layer(conv3, 128, zp_flag=False)
conv4_2 = conv_layer(conv4, 128, zp_flag=False)
mp2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv4)
9
conv7 = conv_layer(mp2, 256, zp_flag=False)
conv8 = conv_layer(conv7, 256, zp_flag=False)
conv9 = conv_layer(conv8, 256, zp_flag=False)
mp3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv9)

conv10 = conv_layer(mp3, 256, zp_flag=False)
conv11 = conv_layer(conv10, 256, zp_flag=False)
mp4 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv11)

conv12 = conv_layer(mp4, 256, zp_flag=False)
conv13 = conv_layer(conv12, 256, zp_flag=False)
mp5 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv13)



# dense layers
flt = Flatten()(mp5)

ds1 = Dense(64, activation='relu')(flt)
ds2 = Dense(64, activation='relu')(ds1)
out = Dense(2, activation='softmax')(ds1)

model = Model(inp, out)

In [43]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 512, 512, 3)       0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 510, 510, 64)      1792      
_________________________________________________________________
batch_normalization_73 (Batc (None, 510, 510, 64)      256       
_________________________________________________________________
leaky_re_lu_73 (LeakyReLU)   (None, 510, 510, 64)      0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 508, 508, 64)      36928     
_________________________________________________________________
batch_normalization_74 (Batc (None, 508, 508, 64)      256       
_________________________________________________________________
leaky_re_lu_74 (LeakyReLU)   (None, 508, 508, 64)      0         
__________

In [44]:
model.compile(loss='binary_crossentropy',optimizer = 'adam',metrics=['accuracy',f1])

In [45]:
epochs = 10

In [46]:
model.fit_generator(train_flow,epochs=epochs,verbose=1)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[4000000,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[Node: dense_4/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=8449971, _device="/job:localhost/replica:0/task:0/device:CPU:0"](dense_1/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op u'dense_4/random_uniform/RandomUniform', defined at:
  File "/cbica/software/external/python/anaconda/2/envs/default/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/cbica/software/external/python/anaconda/2/envs/default/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/cbica/software/external/python/anaconda/2/envs/default/lib/python2.7/site-packages/ipykernel_launche

In [29]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))